# Setting up

In [ ]:
# For Google Colab
# !pip install quickdraw ipyplot plotly

In [ ]:
from quickdraw_preprocess_gan import *
import tensorflow as tf

number_of_names = 10
number_of_drawings = max_drawings


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

# Creating and preprocessing dataset

In [ ]:
random_image = random_image(1)
random_image_array = preprocess_image(random_image)
unprocess_array(random_image_array)

loading chair drawings
load complete


In [ ]:
random_names = random_names(number_of_names, seed = 3) # 2
random_names

['dishwasher', 'dragon', 'clarinet', 'fireplace', 'grass']

In [ ]:
random_names_image_dict = image_dict_names(random_names)
random_names_image_dict

downloading dishwasher from https://storage.googleapis.com/quickdraw_dataset/full/binary/dishwasher.bin
download complete
loading dishwasher drawings
load complete
downloading dragon from https://storage.googleapis.com/quickdraw_dataset/full/binary/dragon.bin
download complete
loading dragon drawings
load complete
downloading clarinet from https://storage.googleapis.com/quickdraw_dataset/full/binary/clarinet.bin
download complete
loading clarinet drawings
load complete
loading fireplace drawings
load complete
downloading grass from https://storage.googleapis.com/quickdraw_dataset/full/binary/grass.bin
download complete
loading grass drawings
load complete


{'dishwasher': [<quickdraw.data.QuickDrawing at 0x7fdbe0d48d90>,
 'dragon': [<quickdraw.data.QuickDrawing at 0x7fdbe0cd8a90>,
 'clarinet': [<quickdraw.data.QuickDrawing at 0x7fdbe0cd84c0>,
 'fireplace': [<quickdraw.data.QuickDrawing at 0x7fdbe0c79370>,
 'grass': [<quickdraw.data.QuickDrawing at 0x7fdbe0c79df0>,
  <quickdraw.data.QuickDrawing at 0x7fdbe0c79dc0>]}

In [ ]:
# plot_image_dict_tabs(random_names_image_dict)

In [ ]:
examples, labels = preprocess_image_dict_to_arrays(random_names_image_dict)

In [ ]:
# ipyplot.plot_class_tabs(images = examples, labels = labels, max_imgs_per_tab = 6, img_width = 128)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(examples)
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(28, 28), dtype=tf.float64, name=None)>

In [ ]:
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size = number_of_names * number_of_drawings)
dataset = dataset.repeat(500)
dataset = dataset.batch(128)
dataset = dataset.prefetch(64)
dataset

<PrefetchDataset element_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float64, name=None)>

In [ ]:
dataset_iterator = dataset.as_numpy_iterator()
dataset_iterator

In [ ]:
batch = dataset_iterator.next()
batch

array([[[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 0.95110868,
         0.95727745, 0.84990025],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 0.85743037],
        ...,
        [0.99498561, 1.        , 0.94583518, ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 0.96097278, ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 0.99204118,
         1.        , 1.        ]],

       [[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1. 

In [ ]:
# ipyplot.plot_images(images = batch, max_images = 6, img_width = 128)

# Building GAN

In [ ]:
# Bring in the sequential api for the generator and discriminator
from tensorflow.keras.models import Sequential
# Bring in the layers for the neural network
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D

## Building generator

In [ ]:
def build_generator(): 
    model = Sequential()

    # Takes in random values and reshapes it to 7x7x128
    # Beginnings of a generated image
    model.add(Dense(7*7*128, input_dim=128))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7,7,128)))

    # Upsampling block 1 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))

    # Upsampling block 2 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional block 1
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional block 2
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))

    # Conv layer to get to one channel
    model.add(Conv2D(1, 4, padding='same', activation='sigmoid'))

    return model

In [ ]:
generator = build_generator()

In [ ]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              809088    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 6272)              0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 128)       409728    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                        

In [ ]:
img = generator.predict(np.random.randn(4,128,1))

1/1 [==============================] - 2s 2s/step


In [ ]:
unprocess_array(img[0, :, :, 0])

## Building discriminator

In [ ]:
def build_discriminator(): 
    model = Sequential()
    
    # First Conv Block
    model.add(Conv2D(32, 5, input_shape = (28,28,1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Second Conv Block
    model.add(Conv2D(64, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Third Conv Block
    model.add(Conv2D(128, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Fourth Conv Block
    model.add(Conv2D(256, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Flatten then pass to dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model 

In [ ]:
discriminator = build_discriminator()

In [ ]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 24, 24, 32)        0         
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 20, 64)        51264     
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 20, 20, 64)        0         
                                                                 
 dropout_1 (Dropout)         (None, 20, 20, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 128)      

In [ ]:
discriminator.predict(img)

1/1 [==============================] - 0s 104ms/step


array([[0.4928986 ],
       [0.49278188],
       [0.49273473],
       [0.49281856]], dtype=float32)

# Training GAN

## Preparing

In [ ]:
# Adam is going to be the optimizer for both
from tensorflow.keras.optimizers import Adam
# Binary cross entropy is going to be the loss for both 
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
g_opt = Adam(learning_rate=0.0001) 
d_opt = Adam(learning_rate=0.00001) 
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [ ]:
# Importing the base model class to subclass our training step 
from tensorflow.keras.models import Model

In [ ]:
class GAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Create attributes for gen and disc
        self.generator = generator 
        self.discriminator = discriminator 
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        # Compile with base class
        super().compile(*args, **kwargs)
        
        # Create attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, batch):
        # Get the data 
        real_images = batch
        fake_images = self.generator(tf.random.normal((128, 128, 1)), training=False)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            # Create labels for real and fakes images
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            # Add some noise to the TRUE outputs
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation - nn learn 
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        # Train the generator 
        with tf.GradientTape() as g_tape: 
            # Generate some new images
            gen_images = self.generator(tf.random.normal((128,128,1)), training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_images, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [ ]:
# Create instance of subclassed model
gan = GAN(generator, discriminator)

In [ ]:
# Compile the model
gan.compile(g_opt, d_opt, g_loss, d_loss)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim,1))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('images', f'generated_img_{epoch}_{i}.png'))

In [ ]:
image_dir = 'images'

if not os.path.exists(image_dir):
    os.mkdir(image_dir)

## Training

In [ ]:
# Recommend 2000 epochs
history = gan.fit(dataset, epochs = 500, callbacks=[ModelMonitor()])

Epoch 1/1000
196/196 [==============================] - 11s 30ms/step - d_loss: 0.6967 - g_loss: 0.7136
Epoch 2/1000
196/196 [==============================] - 6s 30ms/step - d_loss: 0.6922 - g_loss: 0.7205
Epoch 3/1000
196/196 [==============================] - 6s 29ms/step - d_loss: 0.6891 - g_loss: 0.7342
Epoch 4/1000
196/196 [==============================] - 6s 30ms/step - d_loss: 0.6890 - g_loss: 0.7474
Epoch 5/1000
196/196 [==============================] - 6s 29ms/step - d_loss: 0.6828 - g_loss: 0.7819
Epoch 6/1000
196/196 [==============================] - 6s 30ms/step - d_loss: 0.6460 - g_loss: 0.9794
Epoch 7/1000
196/196 [==============================] - 6s 29ms/step - d_loss: 0.5191 - g_loss: 2.0419
Epoch 8/1000
196/196 [==============================] - 6s 30ms/step - d_loss: 0.4002 - g_loss: 3.3617
Epoch 9/1000
196/196 [==============================] - 6s 30ms/step - d_loss: 0.3614 - g_loss: 3.3587
Epoch 10/1000
196/196 [==============================] - 6s 30ms/step - 

KeyboardInterrupt: ignored

# Saving models

In [ ]:
generator.save('generator_2.h5')
discriminator.save('discriminator_2.h5')

In [ ]:
from google.colab import files

files.download('generator_2.h5')
files.download('discriminator_2.h5')

In [ ]:
generator = build_generator()

In [ ]:
generator.load_weights('generator_2.h5')

In [ ]:
!zip -r images.zip images/

In [ ]:
files.download('images.zip')

# Plotting

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots()

# Add traces
fig.add_trace(
    go.Scatter(y = history.history['d_loss'], name = 'd_loss'),
    secondary_y = False
)

fig.add_trace(
    go.Scatter(y = history.history['g_loss'], name = 'g_loss'),
    secondary_y = False
)

# Add figure title
fig.update_layout(
    title_text = 'Loss of GAN Model'
)

# Set x-axis title
fig.update_xaxes(title_text = 'Epoch')

# Testing model

In [ ]:
imgs = generator.predict(tf.random.normal((1, 128, 1)))

In [ ]:
array_to_img(imgs[0])